In [1]:
import sqlalchemy
import pandas as pd
import mysql
from sqlalchemy.sql import text

In [2]:
# 讀取資料
queue = pd.read_parquet('./data_population/queue.parquet', engine='fastparquet')
orderinside = pd.read_parquet('./data_population/orderinside.parquet', engine='fastparquet')
orderoutside = pd.read_parquet('./data_population/orderoutside.parquet', engine='fastparquet')
order_achievement = pd.read_parquet('./data_population/order_achievement.parquet', engine='fastparquet')
base = pd.read_parquet('./data_population/base.parquet', engine='fastparquet')

In [3]:
base.head()

,Serial_Number,Queue_Time,Adult_Count,Kid_Count,Total_Count,Wait_Time,Meal_Time
0,201601010102,2016-01-01 11:05:38,2.0,0.0,2.0,7.333333,32.133333
1,201601010115,2016-01-01 11:09:30,2.0,0.0,2.0,16.600000,30.416667
2,201601010123,2016-01-01 11:08:50,2.0,0.0,2.0,12.616667,27.133333
3,201601010126,2016-01-01 11:09:14,2.0,0.0,2.0,15.683333,45.100000
4,201601010136,2016-01-01 11:15:17,2.0,0.0,2.0,26.233333,41.900000


In [4]:
# # read xlsx
# file_path = "./data/OrderInside.xlsx"
# # df = pd.read_excel(file_path, sheet_name='2016內用訂單')
# df1 = pd.read_excel(file_path, sheet_name='2016內用訂單')
# df2 = pd.read_excel(file_path, sheet_name='2017內用訂單')
# df = pd.concat([df1,df2],axis=0, ignore_index=True)

In [5]:
base_1 = orderinside[['Serial_Number','Queue_Time','Enter_Time','Nation_Code','Adult_Count','Kid_Count', 'Team_Flag', 'Unhandy_Flag']]
base_1['Serial_Number'] = base_1['Serial_Number'].astype(str).str.replace('\.0','')
base_1['Team_Flag'] = base_1['Team_Flag'].astype(bool)
base_1['Unhandy_Flag'] = base_1['Unhandy_Flag'].astype(bool)

# 篩選Queue_Time不為null的資料
# base = base[base['Queue_Time'].notnull()]
base_1 = base_1[base_1['Serial_Number'].isin(base['Serial_Number'])]
# # Queue_Time改變型別 datetime64[ns]
base_1['Date_time'] = base_1['Queue_Time'].astype('datetime64[ns]')
# Nation_Code_1改變型別 'str'
# base['Nation_Code_1'] = base['Nation_Code'].astype('int').astype('str')
base_1['Nation_Code_1'] = base_1['Nation_Code'].fillna(0).astype('int').astype('str')


<ipython-input-5-1d8b9097e8fd>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  base_1['Serial_Number'] = base_1['Serial_Number'].astype(str).str.replace('\.0','')
<ipython-input-5-1d8b9097e8fd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_1['Serial_Number'] = base_1['Serial_Number'].astype(str).str.replace('\.0','')
<ipython-input-5-1d8b9097e8fd>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_1['Team_Flag'] = base_1['Team_Flag'].astype(bool)

In [6]:
base_1['Nation_Code'].fillna(0).astype('int').astype('str')

90          6
101         2
109         2
111         2
119         6
           ..
1025492     5
1025493    71
1025495     6
1025496     5
1025498     1
Name: Nation_Code, Length: 71182, dtype: object

In [7]:
# # 篩選Queue_Time不為null的資料
# base = base[base['Queue_Time'].notnull()]
# base_1.head()

In [8]:
# # Queue_Time改變型別 datetime64[ns]
# base['Date_time'] = base['Queue_Time'].astype('datetime64[ns]')

In [9]:
base_1['Diff_Queue_Enter'] = base_1['Enter_Time'] - base_1['Date_time']
base_1['Diff_Queue_Enter'] = base_1['Diff_Queue_Enter'].dt.total_seconds()/60

In [10]:
# A01:開始排隊時段(小時) Start queuing period hours
base_1['A01_start_queue_hours'] = base_1['Date_time'].dt.hour

In [11]:
# base['Date_time'].iloc[1].dt.time
base_1['Date_time'].iloc[1].strftime("%H:%M:%S") >= '11:00:00'

True

In [12]:
# A02:開始排隊時段別 Start queuing interval
# a:11:00~14:00
# b:14:01~17:00
# c:17:01~關門
# d:其他

def flag_df(df):
    df_time = df['Date_time'].strftime("%H:%M:%S")
    if (df_time >= '11:00:00') and (df_time <= '14:00:00'):
        return 'a'
    elif (df_time > '14:00:00') and (df_time <= '17:00:00'):
        return 'b'
    elif (df_time > '17:00:00'):
        return 'c'
    else:
        return 'd'
    
base_1['A02_queue_interval'] = base_1.apply(flag_df, axis = 1)


In [13]:
# A03:開始排隊時段(小時)是否為用餐時段
# 尖峰 1130-1430 1700-1930 
# 離峰 1430-1700

def flag_df(df):
    df_time = df['Date_time'].strftime("%H:%M:%S")
    if (df_time >= '11:30:00') and (df_time <= '14:30:00'):
        return 'Y'
    elif (df_time >= '17:00:00') and (df_time <= '19:30:00'):
        return 'Y'
    else:
        return 'N'
    
base_1['A03_queue_at_meal_period_bool'] = base_1.apply(flag_df, axis = 1)


In [14]:
# A04:用餐星期

# df['Weekday'] = df['Date'].dt.strftime("%A")
# %A -Full weekday name like MONDAY, TUESDAY etc
# %w -Weekday as a decimal number like 1,2,3 etc
# %a -Abbreviated weekday name like SUN,MON etc
# %Y -year
# %m -month
# %d -day
# %H -hours
# %M -minutes
# %S -seconds
# base['Date_time'].dt.strftime("%A")
base_1['A04_queue_weekday'] = base_1['Date_time'].dt.strftime("%w")

In [15]:
# print(base_1['Date_time'].iloc[1].strftime("%A"))
# print(base_1['Date_time'].iloc[1].strftime("%w"))

In [16]:
# A05:是否為週末用餐
def flag_df(df):
    Is_Weekend = df['Date_time'].strftime("%w")
    while int(Is_Weekend) > 5 :
        return 'Y'
    else:
        return 'N'
base_1['A05_queue_weekend_bool'] = base_1.apply(flag_df, axis = 1)

In [17]:
# A06:用餐月份  
base_1['A06_queue_month'] = base_1['Date_time'].dt.month

In [18]:
# A07:用餐季節
# a:春季(3~5月)
# b:夏季(6~8月)
# c:秋季(9~11月)
# d:冬季(12~2月)
# df_month = base['Date_time'].iloc[1].strftime("%m")
# df_month

def flag_df(df):
    df_month = df['Date_time'].strftime("%m")
    if (df_month in ['03','04','05']):
        return 'a' # 'spring'
    elif (df_month in ['06','07','08']):
        return 'b' # 'summer'
    elif (df_month in ['09','10','11']):
        return 'c' # 'fall'
    elif (df_month in ['12','01','02']):
        return 'd' # 'winter'

base_1['A07_queue_season'] = base_1.apply(flag_df, axis = 1)

In [19]:
# A08:用餐高峰月份
# (2,3,10,11,12)

def flag_df(df):
    df_month = df['Date_time'].strftime("%m")
    if (df_month in ['02','03','10','11','12']):
        return 'Y'
    else:
        return 'N'
    
base_1['A08_queue_at_peak_month_bool'] = base_1.apply(flag_df, axis = 1)


In [20]:
# A10:本次用餐總人數
# 總人數/大人數/小孩數
base_1['A10_total_people_of_diging'] = base_1['Adult_Count'] + base_1['Kid_Count']

In [21]:
# # A11:本次用餐是否有外國人
# # 外國人/本國人/日本人/大陸人/港澳人/韓國人/南洋人/美洲人/紐澳人/歐洲人/亞洲人/大陸+港澳人
# # 1	本地
# # 2	日本
# # 3	大陸
# # 4	港澳
# # 5	韓國
# # 6	南洋
# # 7	美洲
# # 8	紐澳
# # 9	歐洲

# def flag_df(df):
#     # df_dict = list(df['Nation_Code_1'])
#     df_dict = dict(pd.Series(list(df['Nation_Code_1'])).value_counts())
#     Nation_foreign = [str(i) for i in range(2,10)] # ['2', '3', '4', '5', '6', '7', '8', '9']
#     len_Nation = len(df_dict)
#     flag_Nation_foreign = 'N'
#     flag_Nation_1 = 'N'
#     flag_Nation_2 = 'N'
#     flag_Nation_3 = 'N'
#     flag_Nation_4 = 'N'
#     flag_Nation_5 = 'N'
#     flag_Nation_6 = 'N'
#     flag_Nation_7 = 'N'
#     flag_Nation_8 = 'N'
#     flag_Nation_9 = 'N'
#     flag_Nation_ASIA = 'N'
#     flag_Nation_3_4 = 'N'
        
#     cnt_Nation = 0
#     cnt_Nation_foreign = 0
#     for key in df_dict.keys():
#         if key in Nation_foreign:
#             flag_Nation_foreign = 'Y'
#             cnt_Nation_foreign += 1
#         else:
#             cnt_Nation += 1
        
#         if key == '1':
#             flag_Nation_1 = 'Y'
#         elif key == '2':
#             flag_Nation_2 = 'Y'
#         elif key == '3':
#             flag_Nation_3 = 'Y'
#         elif key == '4':
#             flag_Nation_4 = 'Y'
#         elif key == '5':
#             flag_Nation_5 = 'Y'
#         elif key == '6':
#             flag_Nation_6 = 'Y'
#         elif key == '7':
#             flag_Nation_7 = 'Y'
#         elif key == '8':
#             flag_Nation_8 = 'Y'
#         elif key == '9':
#             flag_Nation_9 = 'Y'

#         if key == '3' or key == '4':
#             flag_Nation_3_4 = 'Y'
#         if key in ['1','2', '3', '4', '5']:
#             flag_Nation_ASIA = 'Y'
        
#     return flag_Nation_foreign,flag_Nation_1
# #     return flag_Nation_foreign,cnt_Nation_foreign,cnt_Nation,len_Nation

# base_1['A11_foreigner_dining_bool'] = base_1.apply(flag_df, axis = 1)


In [22]:
# A11:本次用餐是否有外國人
# 外國人/本國人/日本人/大陸人/港澳人/韓國人/南洋人/美洲人/紐澳人/歐洲人/亞洲人/大陸+港澳人
# 1	本地
# 2	日本
# 3	大陸
# 4	港澳
# 5	韓國
# 6	南洋
# 7	美洲
# 8	紐澳
# 9	歐洲
# 'A11_flag_Nation_foreign',外國人
# 'A11_flag_Nation_1',本國人
# 'A11_flag_Nation_2',日本人
# 'A11_flag_Nation_3',大陸人
# 'A11_flag_Nation_4',港澳人
# 'A11_flag_Nation_5',韓國人
# 'A11_flag_Nation_6',南洋人
# 'A11_flag_Nation_7',美洲人
# 'A11_flag_Nation_8',紐澳人
# 'A11_flag_Nation_9',歐洲人
# 'A11_flag_Nation_3_4',大陸+港澳人亞洲人
# 'A11_flag_Nation_ASIA',亞洲人

def flag_df(df):
    # df_dict = list(df['Nation_Code_1'])
    df_dict = dict(pd.Series(list(df['Nation_Code_1'])).value_counts())
    Nation_foreign = [str(i) for i in range(2,10)] # ['2', '3', '4', '5', '6', '7', '8', '9']
    len_Nation = len(df_dict)
    values = dict()
    values['flag_Nation_foreign'] = 'N'
    values['flag_Nation_1'] = 'N'
    values['flag_Nation_2'] = 'N'
    values['flag_Nation_3'] = 'N'
    values['flag_Nation_4'] = 'N'
    values['flag_Nation_5'] = 'N'
    values['flag_Nation_6'] = 'N'
    values['flag_Nation_7'] = 'N'
    values['flag_Nation_8'] = 'N'
    values['flag_Nation_9'] = 'N'
    values['flag_Nation_ASIA'] = 'N'
    values['flag_Nation_3_4'] = 'N'
        
    values['cnt_Nation'] = 0
    values['cnt_Nation_foreign'] = 0
    for key in df_dict.keys():
        if key in Nation_foreign:
            values['flag_Nation_foreign'] = 'Y'
            values['cnt_Nation_foreign'] += 1
        else:
            values['cnt_Nation'] += 1
        
        if key == '1':
            values['flag_Nation_1'] = 'Y'
        elif key == '2':
            values['flag_Nation_2'] = 'Y'
        elif key == '3':
            values['flag_Nation_3'] = 'Y'
        elif key == '4':
            values['flag_Nation_4'] = 'Y'
        elif key == '5':
            values['flag_Nation_5'] = 'Y'
        elif key == '6':
            values['flag_Nation_6'] = 'Y'
        elif key == '7':
            values['flag_Nation_7'] = 'Y'
        elif key == '8':
            values['flag_Nation_8'] = 'Y'
        elif key == '9':
            values['flag_Nation_9'] = 'Y'

        if key == '3' or key == '4':
            values['flag_Nation_3_4'] = 'Y'
        if key in ['1','2', '3', '4', '5']:
            values['flag_Nation_ASIA'] = 'Y'
        
#     return flag_Nation_foreign,flag_Nation_1
    return values
#     return flag_Nation_foreign,cnt_Nation_foreign,cnt_Nation,len_Nation

# base_1['A11_foreigner_dining_bool'] = base_1.apply(flag_df, axis = 1)


In [23]:
def func(df, name):
    df_dict = df
    Name = name
    value = df_dict.get(name)
    return value


In [24]:
base_1['A11_foreigner_dining_dict'] = base_1.apply(flag_df, axis = 1).values #['flag_Nation_foreign']

In [26]:
base_1['A11_flag_Nation_foreign'] = base_1.apply(lambda x: func(x['A11_foreigner_dining_dict'], name = 'flag_Nation_foreign'), axis=1)
base_1['A11_flag_Nation_1'] = base_1.apply(lambda x: func(x['A11_foreigner_dining_dict'], name = 'flag_Nation_1'), axis=1)
base_1['A11_flag_Nation_2'] = base_1.apply(lambda x: func(x['A11_foreigner_dining_dict'], name = 'flag_Nation_2'), axis=1)
base_1['A11_flag_Nation_3'] = base_1.apply(lambda x: func(x['A11_foreigner_dining_dict'], name = 'flag_Nation_3'), axis=1)
base_1['A11_flag_Nation_4'] = base_1.apply(lambda x: func(x['A11_foreigner_dining_dict'], name = 'flag_Nation_4'), axis=1)
base_1['A11_flag_Nation_5'] = base_1.apply(lambda x: func(x['A11_foreigner_dining_dict'], name = 'flag_Nation_5'), axis=1)
base_1['A11_flag_Nation_6'] = base_1.apply(lambda x: func(x['A11_foreigner_dining_dict'], name = 'flag_Nation_6'), axis=1)
base_1['A11_flag_Nation_7'] = base_1.apply(lambda x: func(x['A11_foreigner_dining_dict'], name = 'flag_Nation_7'), axis=1)
base_1['A11_flag_Nation_8'] = base_1.apply(lambda x: func(x['A11_foreigner_dining_dict'], name = 'flag_Nation_8'), axis=1)
base_1['A11_flag_Nation_9'] = base_1.apply(lambda x: func(x['A11_foreigner_dining_dict'], name = 'flag_Nation_9'), axis=1)
base_1['A11_flag_Nation_3_4'] = base_1.apply(lambda x: func(x['A11_foreigner_dining_dict'], name = 'flag_Nation_3_4'), axis=1)
base_1['A11_flag_Nation_ASIA'] = base_1.apply(lambda x: func(x['A11_foreigner_dining_dict'], name = 'flag_Nation_ASIA'), axis=1)
# data['BMI'] = data.apply(lambda x: calc_bmi(x['Weight'], x['Height']), axis=1)

In [27]:
base_1.iloc[14]

Serial_Number                                                         201601010176
Queue_Time                                                     2016-01-01 11:16:41
Enter_Time                                                     2016-01-01 11:45:15
Nation_Code                                                                     71
Adult_Count                                                                    2.0
Kid_Count                                                                      0.0
Team_Flag                                                                    False
Unhandy_Flag                                                                 False
Date_time                                                      2016-01-01 11:16:41
Nation_Code_1                                                                   71
Diff_Queue_Enter                                                         28.566667
A01_start_queue_hours                                                           11
A02_

In [28]:
base_1['A11_foreigner_dining_dict']

90         {'flag_Nation_foreign': 'Y', 'flag_Nation_1': ...
101        {'flag_Nation_foreign': 'Y', 'flag_Nation_1': ...
109        {'flag_Nation_foreign': 'Y', 'flag_Nation_1': ...
111        {'flag_Nation_foreign': 'Y', 'flag_Nation_1': ...
119        {'flag_Nation_foreign': 'Y', 'flag_Nation_1': ...
                                 ...                        
1025492    {'flag_Nation_foreign': 'Y', 'flag_Nation_1': ...
1025493    {'flag_Nation_foreign': 'Y', 'flag_Nation_1': ...
1025495    {'flag_Nation_foreign': 'Y', 'flag_Nation_1': ...
1025496    {'flag_Nation_foreign': 'Y', 'flag_Nation_1': ...
1025498    {'flag_Nation_foreign': 'N', 'flag_Nation_1': ...
Name: A11_foreigner_dining_dict, Length: 71182, dtype: object

In [ ]:
# base_1['A11_foreigner_dining_dict'].iloc[13].get('flag_Nation_foreign')

In [ ]:
# func(base_1['A11_foreigner_dining_dict'].iloc[13],name = 'flag_Nation_foreign')

In [ ]:
# flag_Nation_foreign = 'Y'
# flag_Nation_1 = 'N'
# flag_Nation_2 = 'N'
# flag_Nation_3 = 'N'
# flag_Nation_4 = 'N'
# flag_Nation_5 = 'N'
# flag_Nation_6 = 'N'
# flag_Nation_7 = 'N'
# flag_Nation_8 = 'N'
# flag_Nation_9 = 'N'
# flag_Nation_ASIA = 'N'
# flag_Nation_3_4 = 'N'
# Nation_foreign = [str(i) for i in range(2,10)] # ['2', '3', '4', '5', '6', '7', '8', '9']
# for key in df_dict.keys():
#     if key == '1':
#         flag_Nation_1 = 'Y'
#     elif key == '2':
#         flag_Nation_2 = 'Y'
#     elif key == '3':
#         flag_Nation_3 = 'Y'
#     elif key == '4':
#         flag_Nation_4 = 'Y'
#     elif key == '5':
#         flag_Nation_5 = 'Y'
#     elif key == '6':
#         flag_Nation_6 = 'Y'
#     elif key == '7':
#         flag_Nation_7 = 'Y'
#     elif key == '8':
#         flag_Nation_8 = 'Y'
#     elif key == '9':
#         flag_Nation_9 = 'Y'
    
#     if key == '3' or key == '4':
#         flag_Nation_3_4 = 'Y'
#     if key in ['1','2', '3', '4', '5']:
#         flag_Nation_ASIA = 'Y'
        
#     if key in Nation_foreign:
#         flag_Nation_foreign = 'Y'
#     print(key)
# print('-----------------')
# print(flag_Nation_foreign)
# print(flag_Nation_1)
# print(flag_Nation_2)
# print(flag_Nation_3)
# print(flag_Nation_4)
# print(flag_Nation_5)
# print(flag_Nation_6)
# print(flag_Nation_7)
# print(flag_Nation_8)
# print(flag_Nation_9)
# print(flag_Nation_3_4)
# print(flag_Nation_ASIA)

In [29]:
# A12:本次用餐國別數
def flag_df(df):
#     df_dict = list(df['Nation_Code_1'])
    df_dict = dict(pd.Series(list(df['Nation_Code_1'])).value_counts())
    Nation_foreign = [str(i) for i in range(2,10)] # ['2', '3', '4', '5', '6', '7', '8', '9']
    len_Nation = len(df_dict)
    flag_Nation_foreign = 'N'
    cnt_Nation = 0
    cnt_Nation_foreign = 0
    for key in df_dict.keys():
        if key in Nation_foreign:
            flag_Nation_foreign = 'Y'
            cnt_Nation_foreign += 1
        else:
            cnt_Nation += 1
    return len_Nation
#     return flag_Nation_foreign,cnt_Nation_foreign,cnt_Nation,len_Nation

base_1['A12_dining_countries'] = base_1.apply(flag_df, axis = 1)


In [30]:
# A13:本次用餐非本國的國別數
def flag_df(df):
#     df_dict = list(df['Nation_Code_1'])
    df_dict = dict(pd.Series(list(df['Nation_Code_1'])).value_counts())
    Nation_foreign = [str(i) for i in range(2,10)] # ['2', '3', '4', '5', '6', '7', '8', '9']
    len_Nation = len(df_dict)
    flag_Nation_foreign = 'N'
    cnt_Nation = 0
    cnt_Nation_foreign = 0
    for key in df_dict.keys():
        if key in Nation_foreign:
            flag_Nation_foreign = 'Y'
            cnt_Nation_foreign += 1
        else:
            cnt_Nation += 1
    return cnt_Nation_foreign
#     return flag_Nation_foreign,cnt_Nation_foreign,cnt_Nation,len_Nation

base_1['A13_count_dining_foreign_countries'] = base_1.apply(flag_df, axis = 1)


In [31]:
# A14:本次用餐是否團體
def flag_df(df):
    Is_Team = df['Team_Flag']
    if Is_Team == 1 :
        return 'Y'
    else:
        return 'N'
base_1['A14_team_flag_bool'] = base_1.apply(flag_df, axis = 1)

In [32]:
# A15:次用餐是否博愛
def flag_df(df):
    Is_Unhandy = df['Unhandy_Flag']
    if Is_Unhandy == 1 :
        return 'Y'
    else:
        return 'N'
base_1['A15_unhandy_flag_bool'] = base_1.apply(flag_df, axis = 1)

In [33]:
# 檢視資料
# base_1.head(10)

In [34]:
# base_1.tail(10)

In [35]:
F_base = base_1.copy()

In [36]:
F_base.head()

,Serial_Number,Queue_Time,Enter_Time,Nation_Code,Adult_Count,Kid_Count,Team_Flag,Unhandy_Flag,Date_time,Nation_Code_1,...,A11_flag_Nation_6,A11_flag_Nation_7,A11_flag_Nation_8,A11_flag_Nation_9,A11_flag_Nation_3_4,A11_flag_Nation_ASIA,A12_dining_countries,A13_count_dining_foreign_countries,A14_team_flag_bool,A15_unhandy_flag_bool
90,201601010102,2016-01-01 11:05:38,2016-01-01 11:12:58,6,2.0,0.0,False,False,2016-01-01 11:05:38,6,...,Y,N,N,N,N,N,1,1,N,N
101,201601010115,2016-01-01 11:09:30,2016-01-01 11:26:06,2,2.0,0.0,False,False,2016-01-01 11:09:30,2,...,N,N,N,N,N,Y,1,1,N,N
109,201601010123,2016-01-01 11:08:50,2016-01-01 11:21:27,2,2.0,0.0,False,False,2016-01-01 11:08:50,2,...,N,N,N,N,N,Y,1,1,N,N
111,201601010126,2016-01-01 11:09:14,2016-01-01 11:24:55,2,2.0,0.0,False,False,2016-01-01 11:09:14,2,...,N,N,N,N,N,Y,1,1,N,N
119,201601010136,2016-01-01 11:15:17,2016-01-01 11:41:31,6,2.0,0.0,False,False,2016-01-01 11:15:17,6,...,Y,N,N,N,N,N,1,1,N,N


In [37]:
F_base.columns

Index(['Serial_Number', 'Queue_Time', 'Enter_Time', 'Nation_Code',
       'Adult_Count', 'Kid_Count', 'Team_Flag', 'Unhandy_Flag', 'Date_time',
       'Nation_Code_1', 'Diff_Queue_Enter', 'A01_start_queue_hours',
       'A02_queue_interval', 'A03_queue_at_meal_period_bool',
       'A04_queue_weekday', 'A05_queue_weekend_bool', 'A06_queue_month',
       'A07_queue_season', 'A08_queue_at_peak_month_bool',
       'A10_total_people_of_diging', 'A11_foreigner_dining_dict',
       'A11_flag_Nation_foreign', 'A11_flag_Nation_1', 'A11_flag_Nation_2',
       'A11_flag_Nation_3', 'A11_flag_Nation_4', 'A11_flag_Nation_5',
       'A11_flag_Nation_6', 'A11_flag_Nation_7', 'A11_flag_Nation_8',
       'A11_flag_Nation_9', 'A11_flag_Nation_3_4', 'A11_flag_Nation_ASIA',
       'A12_dining_countries', 'A13_count_dining_foreign_countries',
       'A14_team_flag_bool', 'A15_unhandy_flag_bool'],
      dtype='object')

In [38]:
base_1 = F_base[['Serial_Number','A01_start_queue_hours',
       'A02_queue_interval', 'A03_queue_at_meal_period_bool',
       'A04_queue_weekday', 'A05_queue_weekend_bool', 'A06_queue_month',
       'A07_queue_season', 'A08_queue_at_peak_month_bool',
       'A10_total_people_of_diging',
       'A11_flag_Nation_foreign', 'A11_flag_Nation_1', 'A11_flag_Nation_2',
       'A11_flag_Nation_3', 'A11_flag_Nation_4', 'A11_flag_Nation_5',
       'A11_flag_Nation_6', 'A11_flag_Nation_7', 'A11_flag_Nation_8',
       'A11_flag_Nation_9', 'A11_flag_Nation_3_4', 'A11_flag_Nation_ASIA',
       'A12_dining_countries', 'A13_count_dining_foreign_countries',
       'A14_team_flag_bool', 'A15_unhandy_flag_bool']]

In [39]:
base_1

,Serial_Number,A01_start_queue_hours,A02_queue_interval,A03_queue_at_meal_period_bool,A04_queue_weekday,A05_queue_weekend_bool,A06_queue_month,A07_queue_season,A08_queue_at_peak_month_bool,A10_total_people_of_diging,...,A11_flag_Nation_6,A11_flag_Nation_7,A11_flag_Nation_8,A11_flag_Nation_9,A11_flag_Nation_3_4,A11_flag_Nation_ASIA,A12_dining_countries,A13_count_dining_foreign_countries,A14_team_flag_bool,A15_unhandy_flag_bool
90,201601010102,11,a,N,5,N,1,d,N,2.0,...,Y,N,N,N,N,N,1,1,N,N
101,201601010115,11,a,N,5,N,1,d,N,2.0,...,N,N,N,N,N,Y,1,1,N,N
109,201601010123,11,a,N,5,N,1,d,N,2.0,...,N,N,N,N,N,Y,1,1,N,N
111,201601010126,11,a,N,5,N,1,d,N,2.0,...,N,N,N,N,N,Y,1,1,N,N
119,201601010136,11,a,N,5,N,1,d,N,2.0,...,Y,N,N,N,N,N,1,1,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1025492,201812311500,20,c,N,1,N,12,d,Y,3.0,...,N,N,N,N,N,Y,1,1,N,N
1025493,201812311501,19,c,N,1,N,12,d,Y,2.0,...,N,Y,N,N,N,Y,2,1,N,N
1025495,201812311503,19,c,Y,1,N,12,d,Y,4.0,...,Y,N,N,N,N,N,1,1,N,N
1025496,201812311505,19,c,N,1,N,12,d,Y,2.0,...,N,N,N,N,N,Y,1,1,N,N


In [40]:
# base_1.to_parquet('./data_population/03/F_base_part_01.parquet', engine='fastparquet',index = False)
base_1.to_parquet('./data_population/04/F_base_part_01.parquet', engine='fastparquet',index = False)

In [41]:
base

,Serial_Number,Queue_Time,Adult_Count,Kid_Count,Total_Count,Wait_Time,Meal_Time
0,201601010102,2016-01-01 11:05:38,2.0,0.0,2.0,7.333333,32.133333
1,201601010115,2016-01-01 11:09:30,2.0,0.0,2.0,16.600000,30.416667
2,201601010123,2016-01-01 11:08:50,2.0,0.0,2.0,12.616667,27.133333
3,201601010126,2016-01-01 11:09:14,2.0,0.0,2.0,15.683333,45.100000
4,201601010136,2016-01-01 11:15:17,2.0,0.0,2.0,26.233333,41.900000
...,...,...,...,...,...,...,...
71177,201812311500,2018-12-31 20:23:55,3.0,0.0,3.0,105.566667,46.333333
71178,201812311501,2018-12-31 19:33:48,2.0,0.0,2.0,141.866667,60.383333
71179,201812311503,2018-12-31 19:25:52,4.0,0.0,4.0,143.716667,50.266667
71180,201812311505,2018-12-31 19:39:59,2.0,0.0,2.0,136.416667,41.533333
